In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

chef_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a world-class international chef. You create easy to follow recipes that are loved by millions. You are known for your unique and innovative recipes that are both delicious and healthy and easy to find ingredients to use in your recipes."),
        ("human", "I want to cook {cuisine} food."),
    ]
)

chef_chain = chef_prompt | chat

In [ ]:
veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "you are a vegetarian chef who specialized on making traditional recipies vegetarian. you find alternative ingredients to make the dish vegetarian and healthy. You don't redically modify the reciepe. if there is no alternative for a food just say you don't know how to replace it."),
        ("human", "{recipe}")
    ]
)

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain
# final chain want to attach the recipe value from running the chef_chain. so final chain will pass {cuisine} to chef_chain. and the rerults pass to chat and generate answer. then the answer will attach to {recipe} and pass to veg_chain. vage_chain will pass {recipe} to veg_chef_prompt and pass to chat and generate answer.

final_chain.invoke({"cuisine": "mongolia"})

One popular Mongolian dish that you can try making is Mongolian Beef. It's a flavorful and savory dish that is sure to impress your taste buds. Here's a simple recipe for you to try at home:

Ingredients:
- 1 lb flank steak, thinly sliced
- 2 tablespoons cornstarch
- 2 tablespoons vegetable oil
- 3 cloves garlic, minced
- 1 teaspoon ginger, minced
- 1/2 cup soy sauce
- 1/2 cup water
- 1/2 cup brown sugar
- 1/2 teaspoon red pepper flakes (optional)
- Green onions, chopped for garnish
- Sesame seeds for garnish

Instructions:
1. In a bowl, toss the thinly sliced flank steak with cornstarch until well coated.
2. Heat vegetable oil in a large skillet or wok over medium-high heat. Add the steak slices and cook until browned, about 2-3 minutes per side. Remove the steak from the skillet and set aside.
3. In the same skillet, add a bit more oil if needed and sauté the garlic and ginger until fragrant, about 1 minute.
4. Add soy sauce, water, brown sugar, and red pepper flakes to the skillet. 

AIMessageChunk(content='To make a vegetarian version of Mongolian Beef, we can replace the flank steak with a plant-based alternative such as seitan or tempeh. Here\'s how you can modify the recipe:\n\nIngredients:\n- 1 lb seitan or tempeh, thinly sliced\n- 2 tablespoons cornstarch\n- 2 tablespoons vegetable oil\n- 3 cloves garlic, minced\n- 1 teaspoon ginger, minced\n- 1/2 cup soy sauce\n- 1/2 cup water\n- 1/2 cup brown sugar\n- 1/2 teaspoon red pepper flakes (optional)\n- Green onions, chopped for garnish\n- Sesame seeds for garnish\n\nInstructions:\n1. In a bowl, toss the thinly sliced seitan or tempeh with cornstarch until well coated.\n2. Heat vegetable oil in a large skillet or wok over medium-high heat. Add the seitan or tempeh slices and cook until browned, about 2-3 minutes per side. Remove the seitan or tempeh from the skillet and set aside.\n3. In the same skillet, add a bit more oil if needed and sauté the garlic and ginger until fragrant, about 1 minute.\n4. Add soy sauce,